In [138]:
s1 = """
Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training
on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic
in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of
thousands of examples. By contrast, humans can generally perform a new language task from only
a few examples or from simple instructions – something which current NLP systems still largely
struggle to do. Here we show that scaling up language models greatly improves task-agnostic,
few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-
tuning approaches.
"""
s2 = s1+ """

Specifically, we train GPT-3, an autoregressive language model with 175 billion
parameters, 10x more than any previous non-sparse language model, and test its performance in
the few-shot setting. For all tasks, GPT-3 is applied without any gradient updates or fine-tuning,
with tasks and few-shot demonstrations specified purely via text interaction with the model. GPT-3
achieves strong performance on many NLP datasets, including translation, question-answering, and
cloze tasks, as well as several tasks that require on-the-fly reasoning or domain adaptation, such as
unscrambling words, using a novel word in a sentence, or performing 3-digit arithmetic. At the same
time, we also identify some datasets where GPT-3’s few-shot learning still struggles, as well as some
datasets where GPT-3 faces methodological issues related to training on large web corpora. Finally,
we find that GPT-3 can generate samples of news articles which human evaluators have difficulty
distinguishing from article.

Recent years have featured a trend towards pre-trained language representations in NLP systems, applied in increasingly
flexible and task-agnostic ways for downstream transfer. First, single-layer representations were learned using word
vectors [MCCD13, PSM14] and fed to task-specific architectures, then RNNs with multiple layers of representations
and contextual state were used to form stronger representations [DL15, MBXS17, PNZtY18] (though still applied to
task-specific architectures), and more recently pre-trained recurrent or transformer language models [VSP+17] have
been directly fine-tuned, entirely removing the need for task-specific architectures [RNSS18, DCLT18, HR18].
This last paradigm has led to substantial progress on many challenging NLP tasks such as reading comprehension,
question answering, textual entailment, and many others, and has continued to advance based on new architectures
and algorithms [RSR+19, LOG+19, YDY+19, LCG+19]. However, a major limitation to this approach is that while
the architecture is task-agnostic, there is still a need for task-specific datasets and task-specific fine-tuning: to achieve
strong performance on a desired task typically requires fine-tuning on a dataset of thousands to hundreds of thousands
of examples specific to that task. Removing this limitation would be desirable, for several reasons.
First, from a practical perspective, the need for a large dataset of labeled examples for every new task limits the
applicability of language models. There exists a very wide range of possible useful language tasks, encompassing
anything from correcting grammar, to generating examples of an abstract concept, to critiquing a short story. For many
of these tasks it is difficult to collect a large supervised training dataset, especially when the process must be repeated
for every new task.
Second, the potential to exploit spurious correlations in training data fundamentally grows with the expressiveness
of the model and the narrowness of the training distribution. This can create problems for the pre-training plus
fine-tuning paradigm, where models are designed to be large to absorb information during pre-training, but are then
fine-tuned on very narrow task distributions. For instance [HLW+20] observe that larger models do not necessarily
generalize better out-of-distribution. There is evidence that suggests that the generalization achieved under this paradigm
can be poor because the model is overly specific to the training distribution and does not generalize well outside it
[YdC+19, MPL19]. Thus, the performance of fine-tuned models on specific benchmarks, even when it is nominally at
human-level, may exaggerate actual performance on the underlying task [GSL+18, NK19].
Third, humans do not require large supervised datasets to learn most language tasks – a brief directive in natural
language (e.g. “please tell me if this sentence describes something happy or something sad”) or at most a tiny number
of demonstrations (e.g. “here are two examples of people acting brave; please give a third example of bravery”) is often
Figure 1.1: Language model meta-learning. During unsupervised pre-training, a language model develops a broad
set of skills and pattern recognition abilities. It then uses these abilities at inference time to rapidly adapt to or recognize
the desired task. We use the term “in-context learning” to describe the inner loop of this process, which occurs within
the forward-pass upon each sequence. The sequences in this diagram are not intended to be representative of the data a
model would see during pre-training, but are intended to show that there are sometimes repeated sub-tasks embedded
within a single sequence.
3
Figure 1.2: Larger models make increasingly efficient use of in-context information. We show in-context learning
performance on a simple task requiring the model to remove random symbols from a word, both with and without a
natural language task description (see Sec. 3.9.2). The steeper “in-context learning curves” for large models demonstrate
improved ability to learn a task from contextual information. We see qualitatively similar behavior across a wide range
of tasks.
sufficient to enable a human to perform a new task to at least a reasonable degree of competence. Aside from pointing
to a conceptual limitation in our current NLP techniques, this adaptability has practical advantages – it allows humans
to seamlessly mix together or switch between many tasks and skills, for example performing addition during a lengthy
dialogue. To be broadly useful, we would someday like our NLP systems to have this same fluidity and generality.
One potential route towards addressing these issues is meta-learning1 – which in the context of language models means
the model develops a broad set of skills and pattern recognition abilities at training time, and then uses those abilities
at inference time to rapidly adapt to or recognize the desired task (illustrated in Figure 1.1). Recent work [RWC+19]
attempts to do this via what we call “in-context learning”, using the text input of a pretrained language model as a form
of task specification: the model is conditioned on a natural language instruction and/or a few demonstrations of the task
and is then expected to complete further instances of the task simply by predicting what comes next.
While it has shown some initial promise, this approach still achieves results far inferior to fine-tuning – for example
[RWC+19] achieves only 4% on Natural Questions, and even its 55 F1 CoQa result is now more than 35 points behind
the state of the art. Meta-learning clearly requires substantial improvement in order to be viable as a practical method of
solving language tasks.
Another recent trend in language modeling may offer a way forward. In recent years the capacity of transformer
language models has increased substantially, from 100 million parameters [RNSS18], to 300 million parameters
[DCLT18], to 1.5 billion parameters [RWC+19], to 8 billion parameters [SPP+19], 11 billion parameters [RSR+19],
and finally 17 billion parameters [Tur20]. Each increase has brought improvements in text synthesis and/or downstream
NLP tasks, and there is evidence suggesting that log loss, which correlates well with many downstream tasks, follows a
smooth trend of improvement with scale [KMH+20]. Since in-context learning involves absorbing many skills and
tasks within the parameters of the model, it is plausible that in-context learning abilities might show similarly strong
gains with scale.
1In the context of language models this has sometimes been called “zero-shot transfer”, but this term is potentially ambiguous:
the method is “zero-shot” in the sense that no gradient updates are performed, but it often involves providing inference-time
demonstrations to the model, so is not truly learning from zero examples. To avoid this confusion, we use the term “meta-learning”
to capture the inner-loop / outer-loop structure of the general method, and the term “in context-learning” to refer to the inner
loop of meta-learning. We further specialize the description to “zero-shot”, “one-shot”, or “few-shot” depending on how many
demonstrations are provided at inference time. These terms are intended to remain agnostic on the question of whether the model
learns new tasks from scratch at inference time or simply recognizes patterns seen during training – this is an important issue which
we discuss later in the paper, but “meta-learning” is intended to encompass both possibilities, and simply describes the inner-outer
loop structure.
4
Figure 1.3: Aggregate performance for all 42 accuracy-denominated benchmarks While zero-shot performance
improves steadily with model size, few-shot performance increases more rapidly, demonstrating that larger models are
more proficient at in-context learning. See Figure 3.8 for a more detailed analysis on SuperGLUE, a standard NLP
benchmark suite.
In this paper, we test this hypothesis by training a 175 billion parameter autoregressive language model, which we call
GPT-3, and measuring its in-context learning abilities. Specifically, we evaluate GPT-3 on over two dozen NLP datasets,
as well as several novel tasks designed to test rapid adaptation to tasks unlikely to be directly contained in the training
set. For each task, we evaluate GPT-3 under 3 conditions: (a) “few-shot learning”, or in-context learning where we
allow as many demonstrations as will fit into the model’s context window (typically 10 to 100), (b) “one-shot learning”,
where we allow only one demonstration, and (c) “zero-shot” learning, where no demonstrations are allowed and only
an instruction in natural language is given to the model. GPT-3 could also in principle be evaluated in the traditional
fine-tuning setting, but we leave this to future work.
Figure 1.2 illustrates the conditions we study, and shows few-shot learning of a simple task requiring the model to
remove extraneous symbols from a word. Model performance improves with the addition of a natural language task
description, and with the number of examples in the model’s context, K. Few-shot learning also improves dramatically
with model size. Though the results in this case are particularly striking, the general trends with both model size and
number of examples in-context hold for most tasks we study. We emphasize that these “learning” curves involve no
gradient updates or fine-tuning, just increasing numbers of demonstrations given as conditioning.
Broadly, on NLP tasks GPT-3 achieves promising results in the zero-shot and one-shot settings, and in the the few-shot
setting is sometimes competitive with or even occasionally surpasses state-of-the-art (despite state-of-the-art being held
by fine-tuned models). For example, GPT-3 achieves 81.5 F1 on CoQA in the zero-shot setting, 84.0 F1 on CoQA in
the one-shot setting, 85.0 F1 in the few-shot setting. Similarly, GPT-3 achieves 64.3% accuracy on TriviaQA in the
zero-shot setting, 68.0% in the one-shot setting, and 71.2% in the few-shot setting, the last of which is state-of-the-art
relative to fine-tuned models operating in the same closed-book setting.
GPT-3 also displays one-shot and few-shot proficiency at tasks designed to test rapid adaption or on-the-fly reasoning,
which include unscrambling words, performing arithmetic, and using novel words in a sentence after seeing them
defined only once. We also show that in the few-shot setting, GPT-3 can generate synthetic news articles which human
evaluators have difficulty distinguishing from human-generated articles.
At the same time, we also find some tasks on which few-shot performance struggles, even at the scale of GPT-3. This
includes natural language inference tasks like the ANLI dataset, and some reading comprehension datasets like RACE
or QuAC. By presenting a broad characterization of GPT-3’s strengths and weaknesses, including these limitations, we
hope to stimulate study of few-shot learning in language models and draw attention to where progress is most needed.
A heuristic sense of the overall results can be seen in Figure 1.3, which aggregates the various tasks (though it should
not be seen as a rigorous or meaningful benchmark in itself).
5
We also undertake a systematic study of “data contamination” – a growing problem when training high capacity models
on datasets such as Common Crawl, which can potentially include content from test datasets simply because such
content often exists on the web. In this paper we develop systematic tools to measure data contamination and quantify
its distorting effects. Although we find that data contamination has a minimal effect on GPT-3’s performance on most
datasets, we do identify a few datasets where it could be inflating results, and we either do not report results on these
datasets or we note them with an asterisk, depending on the severity.
In addition to all the above, we also train a series of smaller models (ranging from 125 million parameters to 13 billion
parameters) in order to compare their performance to GPT-3 in the zero, one and few-shot settings. Broadly, for most
tasks we find relatively smooth scaling with model capacity in all three settings; one notable pattern is that the gap
between zero-, one-, and few-shot performance often grows with model capacity, perhaps suggesting that larger models
are more proficient meta-learners.
Finally, given the broad spectrum of capabilities displayed by GPT-3, we discuss concerns about bias, fairness, and
broader societal impacts, and attempt a preliminary analysis of GPT-3’s characteristics in this regard.
The remainder of this paper is organized as follows. In Section 2, we describe our approach and methods for training
GPT-3 and evaluating it. Section 3 presents results on the full range of tasks in the zero-, one- and few-shot settings.
Section 4 addresses questions of data contamination (train-test overlap). Section 5 discusses limitations of GPT-3.
Section 6 discusses broader impacts. Section 7 reviews related work and Section 8 concludes.
"""

In [ ]:
from typing import Counter
from collections import deque

class BytePairEncodingTokenizer:
    def __init__(self) -> None:
        self.stats = Counter()
        self.desired_vocab_size = 700
        self.merges: dict[tuple[int,int], int] = {}
        self.vocab = {}
        
    def get_stats(self, tokens: list[int]) -> dict[tuple[int, int], int]:
        stats = Counter()
        for pair in zip(tokens, tokens[1:]):
            stats[pair] += 1
            
        return stats
        
    def train(self, training_string: str, desired_vocab_length: int = 4096):
        enc = training_string.encode('utf-8')
        tokens = list(map(int, enc))

        vocab_size = len(self.get_stats(tokens))
        num_merges = vocab_size - desired_vocab_length
        ids = list(tokens)
        new_token = 256
        for _ in range(num_merges):
            stats = self.get_stats(ids)
            top_pair = max(stats, key=stats.get)
            ids = self.merge(ids, top_pair, replacement=new_token)
            self.merges[top_pair] = new_token
            new_token+=1
            
        for pair, token in self.merges.items():
            self.vocab[token] = pair
        
        return ids
        
    # Will achieve less compression than Kapathy's algo but runs in O(n) instead of O(n^2)
    def encode(self, s: str) -> list[int]:
        tokens = list(map(int, s.encode('utf-8')))
        final_idx = len(tokens)-2
        idx = 0
        while idx < final_idx:
            pair = (tokens[idx], tokens[idx+1])
            if pair in self.merges:
                tokens[idx] = self.merges[pair]
                del tokens[idx+1]
                final_idx-=1
            
            idx+=1
        return tokens
        
        
    def merge(self, input_ids: list[int], pair: tuple[int, int], replacement: int) -> list[int]:
        final_index = len(input_ids)
        idx = 0
        result = []
        while idx < final_index:
            if idx + 1 < final_index and input_ids[idx] == pair[0] and input_ids[idx+1] == pair[1]:
                result.append(replacement)
                idx+=2
            else:
                result.append(input_ids[idx])
                idx+=1
                
        return result
    
    def decode(self,tokens: list[int]) -> str:
        if len(tokens) == 0:
            return ''
        result: list[int] = []
        for token in tokens:
            queue = deque([token])
            while len(queue) > 0:
                current = queue.popleft()
                if current < 256:
                    result.append(current)
                else:
                    new_pair = self.vocab[current]
                    queue.appendleft(new_pair[1])
                    queue.appendleft(new_pair[0])
        return bytes(result).decode('utf-8', errors='replace')
                    
                
        
            
t = BytePairEncodingTokenizer()
ids = t.train(s2, 600)
encoded = t.encode(s1)
print(f'Compression ration: {(len(s1)/ len(encoded)):.2f}x')


ValueError: max() arg is an empty sequence

In [196]:
max(t.vocab.keys())
# len(t.vocab) + 255

421

In [183]:
gpt2pat = re.compile(r)

gpt2pat.findall("Hey beech how are you. you'll hate this. 15+1993=22dumby")

['Hey',
 ' beech',
 ' how',
 ' are',
 ' you',
 '.',
 ' you',
 "'ll",
 ' hate',
 ' this',
 '.',
 ' 15',
 '+',
 '1993',
 '=',
 '22',
 'dumby']

In [151]:
t.encode("""Our basic pre-training approach, including model, data, and training, is similar to the process described in [RWC+19],
with relatively straightforward scaling up of the model size, dataset size and diversity, and length of training. Our use
of in-context learning is also similar to [RWC+19], but in this work we systematically explore different settings for
learning within the context. Therefore, we start this section by explicitly defining and contrasting the different settings
that we will be evaluating GPT-3 on or could in principle evaluate GPT-3 on. These settings can be seen as lying on a
spectrum of how much task-specific data they tend to rely on. Specifically, we can identify at least four points on this
spectrum (see Figure 2.1 for an illustration)""")

[79,
 307,
 32,
 98,
 273,
 105,
 99,
 32,
 112,
 269,
 45,
 116,
 288,
 257,
 257,
 103,
 261,
 112,
 112,
 293,
 97,
 294,
 276,
 257,
 99,
 108,
 117,
 308,
 110,
 103,
 32,
 290,
 274,
 108,
 276,
 100,
 277,
 97,
 276,
 268,
 275,
 116,
 288,
 257,
 257,
 103,
 276,
 105,
 258,
 115,
 105,
 109,
 105,
 108,
 272,
 259,
 286,
 271,
 256,
 112,
 293,
 99,
 311,
 258,
 274,
 115,
 99,
 114,
 105,
 98,
 101,
 275,
 257,
 32,
 91,
 82,
 87,
 67,
 43,
 49,
 57,
 93,
 44,
 10,
 119,
 315,
 104,
 32,
 269,
 108,
 277,
 105,
 118,
 101,
 108,
 281,
 279,
 288,
 105,
 103,
 104,
 116,
 102,
 267,
 119,
 272,
 275,
 115,
 99,
 278,
 257,
 103,
 32,
 117,
 112,
 32,
 280,
 259,
 104,
 256,
 290,
 274,
 108,
 301,
 105,
 122,
 101,
 276,
 100,
 277,
 273,
 292,
 301,
 105,
 122,
 256,
 268,
 275,
 308,
 118,
 262,
 115,
 315,
 121,
 276,
 268,
 275,
 282,
 110,
 103,
 271,
 32,
 280,
 259,
 288,
 257,
 257,
 103,
 289,
 79,
 307,
 32,
 117,
 115,
 101,
 10,
 280,
 32,
 257,
 45,
 99,
 260,
 11